In [1]:
import sqlite3

# Use only the first time
conn = sqlite3.connect("mydatabase.db") # or use :memory: to put it in RAM
 
cursor = conn.cursor()
 
# create a table
cursor.execute("""CREATE TABLE test_twitter
                  (user text, tweet text) 
               """)

In [5]:
# Read keywords to track from track.csv into variable trackstring

import csv
with open('track.csv', newline='') as csvfile:
    trackreader = csv.reader(csvfile, delimiter=';', quotechar='|')
    for row in trackreader:
        trackstring = ', '.join(row)
        print(','.join(row))

BMW, daimler, mercedes benz, AMG, audi


In [6]:
# The main body of the streamer class, writes to SQL

from twython import TwythonStreamer  
import json

# Filter out unwanted data
def process_tweet(tweet):  
    #d = {}
    #d['text'] = tweet['text']
    #d['user'] = tweet['user']['screen_name']
    d = [tweet['text'], tweet['user']['screen_name']]
    return d


# Create a class that inherits TwythonStreamer
class MyStreamer(TwythonStreamer):     

    # Received data
    def on_success(self, data):

        # Only collect tweets in English
        if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            self.save_to_sql(tweet_data)

    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()

    # Save each tweet to csv file
    def save_to_sql(self, tweet):
        #print(list(tweet.values()))
        print(tweet)
        cursor.execute("INSERT INTO test_twitter VALUES (?,?)", tweet)
        conn.commit()

In [7]:
# Get the credentials:
# Load credentials from json file
with open("twitter_credentials.json", "r") as file:  
    creds = json.load(file)

# Instantiate from our streaming class
stream = MyStreamer(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'],  
                    creds['ACCESS_TOKEN_KEY'], creds['ACCESS_TOKEN_SECRET'])
# Start the stream
geo_locations = '-122.75,36.8,-121.75,37.8,-74,40,-73,41'
stream.statuses.filter(track=trackstring,locations=geo_locations)  

['Bruh I’m watching queen of the south and how was Brenda a cartel wife and she keep running her damn mouth!!!!!!!!', 'kimNOberlyyy']
["RT @Shardoninfinity: Good night faaaam, okay lang yan. Di ko din alam bakit amg unfair ni twitter, pero If there's a will there's a way so…", 'SHARDONFOREVER1']
["@RKJ65 So sorry for a profound loss. She lived a good kitty life, but it doesn't take away the sadness and void! I… https://t.co/mwcX0vTwDK", 'lariti2']
['@humptyleo congrats on your graduation!!!!!! https://t.co/iniTJ4sGTa', 'ranchee94']
['#Repost danceloftlb \n・・・\nThe recital is here!… https://t.co/kMJLuW3fwW', 'sharena']
["RT @Telegraph: Mercedes-Benz joins the grubby identity parade of German emissions 'cheats' – to its shame https://t.co/09eL0cnwaZ", 'MagneClip']
["@AudiIN the battery quality of audi is so poor it has been changed twice in 4years. I didn't expect such bad qualit… https://t.co/7K0Z0SPy8y", '07kriti']
['@dannyd1976 add this to the summer bbq mix!  https://t.co/E6PLm5F9Ys'

KeyboardInterrupt: 